# Demo notebook for Model Reader

In [ ]:
# Delete the time files before running.
model = 'MARBLE'
variables_requested = ['rho', 'p']  # one variable from each coordinate
# since all variables are on the same coordinate grid, only one needs to be validated  
# change file path to where data is stored on your machine
file_dir = 'D:/MARBLE/lowresolution/'
from os.path import isfile
from os import remove
if isfile(file_dir+model+'_times.txt'):
    remove(file_dir+model+'_times.txt')
if isfile(file_dir+model+'_list.txt'):
    remove(file_dir+model+'_list.txt')

In [ ]:
# confirm MW functions work appropriately with model varnames dictionary
import kamodo_ccmc.flythrough.model_wrapper as MW
MW.Variable_Search('density')

In [ ]:
MW.Variable_Search('density', model)

In [ ]:
MW.Variable_Search('density', model, file_dir)

In [ ]:
MW.Var_3D(model)

In [ ]:
MW.Var_units(model, variables_requested)

In [ ]:
# Confirm time method works with model reader
MW.File_Times(model, file_dir)

In [ ]:
# Confirm file list method works with model reader
MW.File_List(model, file_dir)

In [ ]:
# Check that time files creation works, that reader works for one variable,
# and that an unknown variable request does not break it.
reader = MW.Model_Reader(model)
kamodo_object = reader(file_dir, variables_requested=['Trash'])
kamodo_object

In [ ]:
# Check that reader works for one variable with an unknown variable
kamodo_object = reader(file_dir, variables_requested=['Trash', variables_requested[0]])
kamodo_object

In [ ]:
# Test model reader with one variable from each coordinate system
kamodo_object = reader(file_dir, variables_requested=variables_requested)
kamodo_object

In [ ]:
# Check that reading the time files works and that reader works for one variable,
kamodo_object = reader(file_dir, variables_requested=variables_requested[:1])
kamodo_object

In [ ]:
# Confirm that interpolation works. 
from math import isnan
print(kamodo_object.rho([5.2, 10., 60., 50.]))
if isnan(kamodo_object.rho([5.2, 10., 60., 50.])[0]):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
print(kamodo_object.rho_ijk(time=5.2, X=10., Y=60., Z=50.))
if isnan(kamodo_object.rho_ijk(time=5.2, X=10., Y=60., Z=50.)):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
if not kamodo_object.rho([5.2, 10., 60., 50.]) == kamodo_object.rho_ijk(time=5.2, X=10., Y=60., Z=50.):
    raise AttributeError('Values are not equal.')
else:
    print('Values are equal.')
print(kamodo_object.rho_ijk(time=5.2, X=10).shape)

In [ ]:
# Check that reading the time files works, and that the reader works for all variables
kamodo_object = reader(file_dir)
kamodo_object

In [ ]:
# Get a list of all of the functionalized variables, both regular and gridded
var_list = list(MW.Variable_Search('', model, file_dir, return_dict=True).keys())
varijk_list = sorted(var_list + [item+'_ijk' for item in var_list])
varijk_list

In [ ]:
# Test coordinate range logic for all variables
MW.Coord_Range(kamodo_object, varijk_list)

In [ ]:
# Check that the kamodo object was built properly.
print(kamodo_object.rho([5.2, 10., 60., 50.]))
if isnan(kamodo_object.rho([5.2, 10., 60., 50.])):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')

In [ ]:
# Check that the reader works for the testing subset
kamodo_object = reader(file_dir, variables_requested=variables_requested)
kamodo_object

In [ ]:
# Confirm that the interpolator works for each testing variable and type
print(kamodo_object.rho([5.2, 10., 60., 50.]))
print(kamodo_object.rho_ijk(time=5.2, X=10., Y=60., Z=50.))
if not kamodo_object.rho([5.2, 10., 60., 50.]) == kamodo_object.rho_ijk(time=5.2, X=10., Y=60., Z=50.):
    raise AttributeError('Values are not equal.')
else:
    print('Values are equal.')
print(kamodo_object.rho_ijk(time=5.2, X=10.).shape)

In [ ]:
# Generate a plot for validation
from kamodo_ccmc.tools.plotfunctions import toColor
toColor(kamodo_object.plot('rho_ijk', plot_partial={'rho_ijk': {'time': 5.2, 'Z': 50.}}),
        colorscale="Viridis")

In [ ]:
# Generate a plot for validation
toColor(kamodo_object.plot('rho_ijk', plot_partial={'rho_ijk': {'X': 10., 'Y': 60.}}),
        colorscale="Viridis")

In [ ]:
# Test that more than one variable works through the flythrough
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
import datetime as dt
# Need to set the datetimes below to match the data
start_utcts = dt.datetime(2013, 5, 31, 15).replace(tzinfo=dt.timezone.utc).timestamp()
end_utcts = dt.datetime(2013, 6, 2, 2).replace(tzinfo=dt.timezone.utc).timestamp()-1
results = SF.ModelFlythrough(model, file_dir, [variables_requested[0]], [start_utcts, end_utcts], [0., 180.], [60., -60.],
                             [400., 400.], 'GDZ-sph')
results[variables_requested[0]]

In [ ]:
# Test that one variable works through the flythrough
results = SF.ModelFlythrough(model, file_dir, [variables_requested[0]], [start_utcts], [0.], [60.],
                             [400.], 'GDZ-sph')
results[variables_requested[0]]